# Satcen Dataset Handling

Perform any processing that may be required on the Satcen dataset.

## Add New Files

Handle the addition of newly-received images without skiffs.

### Rename the Files

In [1]:
import os
import pathlib

# path to satcen_dataset (contains "full" and "original" subfolders)
satcen_path = pathlib.Path('../satcen_dataset/').resolve()

# rename new files
for f in os.listdir(satcen_path/'full/pictures'):

    # check if file has original name
    if 'tudor' in f:
        os.rename(satcen_path/'full/pictures'/f, satcen_path/'full/pictures'/f[6:])

### Construct the Labels

In [2]:
import json

# load the annotations of the original images
file = open(satcen_path/'original/SatCen_skiffs256.json')
labels = json.load(file)['batch']['annotations']

# reformat original annotations
labels = list(map(lambda x: {
    'name': x['name'],
    'bounding_boxes': [box['data'] for box in x['objects']]
}, labels))

# new images do not contain any ships, add bounding_boxes: []
# for the corresponding entry in the labels list
# new images contain "chip" in the beginning of the filename
pictures = os.listdir(satcen_path/'full/pictures')
for f in pictures:

    # new image
    if 'chip' in f:
        labels.append({
            'name': f,
            'bounding_boxes': []
        })

# calculate number of positive and negative observations
no_pos_obs = len(list(filter(lambda x: len(x['bounding_boxes']) > 0, labels)))
no_neg_obs = len(list(filter(lambda x: len(x['bounding_boxes']) == 0, labels)))

# print stats
print(f'Total number of labels: {len(labels)}')
print(f'Positive observations: {no_pos_obs}')
print(f'Negative observations: {no_neg_obs}')

# save json
labels_path = satcen_path/'full/labels.json'

# delete if already exists
if 'labels.json' in os.listdir():
    os.remove(labels_path)

# write to file
with open(labels_path, 'w') as f:
    json.dump(labels, f)

Total number of labels: 9420
Positive observations: 1525
Negative observations: 7895


In [3]:
print(labels[0])

{'name': '272-512-0.png', 'bounding_boxes': [[{'x': 33, 'y': 154}, {'x': 32, 'y': 175}, {'x': 42, 'y': 176}, {'x': 45, 'y': 154}], [{'x': 18, 'y': 124}, {'x': 17, 'y': 147}, {'x': 29, 'y': 149}, {'x': 31, 'y': 129}], [{'x': 67, 'y': 129}, {'x': 54, 'y': 147}, {'x': 61, 'y': 151}, {'x': 74, 'y': 138}], [{'x': 64, 'y': 125}, {'x': 48, 'y': 138}, {'x': 55, 'y': 145}, {'x': 70, 'y': 134}], [{'x': 50, 'y': 112}, {'x': 40, 'y': 131}, {'x': 51, 'y': 138}, {'x': 60, 'y': 119}], [{'x': 43, 'y': 104}, {'x': 32, 'y': 122}, {'x': 42, 'y': 126}, {'x': 52, 'y': 108}], [{'x': 36, 'y': 100}, {'x': 24, 'y': 117}, {'x': 32, 'y': 123}, {'x': 45, 'y': 106}], [{'x': 51, 'y': 196}, {'x': 30, 'y': 196}, {'x': 31, 'y': 208}, {'x': 51, 'y': 208}], [{'x': 31, 'y': 203}, {'x': 9, 'y': 202}, {'x': 9, 'y': 215}, {'x': 32, 'y': 215}], [{'x': 50, 'y': 208}, {'x': 31, 'y': 205}, {'x': 29, 'y': 216}, {'x': 51, 'y': 219}], [{'x': 34, 'y': 217}, {'x': 57, 'y': 220}, {'x': 54, 'y': 233}, {'x': 33, 'y': 229}], [{'x': 31, 

### Remove Images that Are Not Annotated

In [7]:
# extract file names from the labels
annotated_images = list(map(lambda x: x['name'], labels))
print(f'Number of annotations: {len(annotated_images)}')

# loop through the images in the satcen_dataset/full/pictures folder
all_images = os.listdir(satcen_path/'full/pictures')
for img in all_images:

    # check if image is annotated
    if img not in annotated_images:

        # remove image
        os.remove(satcen_path/f'full/pictures/{img}')

print(f'Total number of images (pictures folder): {len(os.listdir(satcen_path/"full/pictures"))}')

Number of annotations: 9420
Total number of images (pictures folder): 9420
